In [1]:
import pandas as pd

datafile = "data_science_student_marks.csv" # change data file when needed
fileType = datafile.split('.')[-1].lower()

if fileType == "csv":
    inputTable = pd.read_csv(datafile)
elif fileType in ["xlsx", "xls"]:
    inputTable = pd.read_excel(datafile,sheet_name=1)

targetDataFrame = inputTable.head(10)

In [2]:
#Randomly Remove Values and create outliers

import numpy as np

numericColumns = inputTable.select_dtypes(include='number').columns.tolist()

#Randomly remove 20% of the data
for column in inputTable:
    randomCells = np.random.rand(len(inputTable)) < 0.2
    inputTable.loc[randomCells, column] = np.nan

#Randomly create outliers of 10% of the data
for column in inputTable.select_dtypes(include='number'):
    randomCells = np.random.rand(len(inputTable)) < 0.1
    inputTable.loc[randomCells, column] = inputTable.loc[randomCells, column] * 100

inputTable.head(50)

,student_id,location,age,sql_marks,excel_marks,python_marks,power_bi_marks,english_marks
0,4.0,Sydney,24.0,95.0,99.0,NaN,NaN,7500.0
1,5.0,Tokyo,24.0,99.0,NaN,89.0,NaN,NaN
2,NaN,Berlin,22.0,72.0,70.0,99.0,79.0,77.0
3,7.0,NaN,NaN,97.0,90.0,74.0,72.0,85.0
4,8.0,NaN,22.0,91.0,71.0,79.0,8000.0,75.0
5,NaN,Toronto,20.0,93.0,88.0,NaN,93.0,72.0
6,10.0,NaN,18.0,77.0,NaN,100.0,98.0,NaN
7,11.0,Toronto,2100.0,78.0,90.0,88.0,7900.0,72.0
8,NaN,Melbourne,24.0,80.0,95.0,87.0,90.0,94.0
9,13.0,London,19.0,97.0,80.0,96.0,NaN,86.0


In [3]:
#Transformation Table

inputTable.columns = [column.lower() for column in inputTable.columns]

numericColumns = []
yesAndNoColumns = []

#Find numeric columns by checking top ten values for numeric data
for column in inputTable:
    topTenValues = inputTable[column].dropna().head(10)
    numericCount = pd.to_numeric(topTenValues, errors='coerce').notna().sum()

    if numericCount > 0:
        numericColumns.append(column)

    yesNoCount = topTenValues.isin(["yes", "no", "Yes","No"]).sum()
    if yesNoCount > 0:
        yesAndNoColumns.append(column)
        
#Get rid of exceptional values
for column in numericColumns:
    inputTable[column] = pd.to_numeric(inputTable[column], errors="coerce")

    #Find the lower quantile of the column values
    lowerQuantile = inputTable[column].quantile(0.25)
    
    #Find the upper quantile of the values in each column
    upperQuantile = inputTable[column].quantile(0.75)
    
    #Find the inter quantile range of the values in each column
    interQuantileRange = upperQuantile - lowerQuantile

    lowerBound = lowerQuantile - 1.5 * interQuantileRange
    upperBound = upperQuantile + 1.5 * interQuantileRange

    #Seperate outliers using the bounds
    outliers = (inputTable[column] < lowerBound) | (inputTable[column] > upperBound)
    
    #Divide outliers by 100
    inputTable.loc[outliers, column] =  inputTable.loc[outliers, column] / 100

for column in yesAndNoColumns:
    inputTable[column] = inputTable[column].map({"yes": 1, "no": 0,"Yes": 1, "No": 0})

inputTable.head(50)

,student_id,location,age,sql_marks,excel_marks,python_marks,power_bi_marks,english_marks
0,4.0,Sydney,24.0,95.0,99.0,NaN,NaN,75.0
1,5.0,Tokyo,24.0,99.0,NaN,89.0,NaN,NaN
2,NaN,Berlin,22.0,72.0,70.0,99.0,79.0,77.0
3,7.0,NaN,NaN,97.0,90.0,74.0,72.0,85.0
4,8.0,NaN,22.0,91.0,71.0,79.0,80.0,75.0
5,NaN,Toronto,20.0,93.0,88.0,NaN,93.0,72.0
6,10.0,NaN,18.0,77.0,NaN,100.0,98.0,NaN
7,11.0,Toronto,21.0,78.0,90.0,88.0,79.0,72.0
8,NaN,Melbourne,24.0,80.0,95.0,87.0,90.0,94.0
9,13.0,London,19.0,97.0,80.0,96.0,NaN,86.0


In [4]:
#Convert numeric columns with no decimals into integers

for column in numericColumns: 
    pureData = inputTable[column].dropna()
    hasDecimals = (pureData.astype(int) != pureData).any()
    if hasDecimals == False:
        inputTable[column] = inputTable[column].astype("Int64")

inputTable.head(50)
#TODO sort out NaN and make yesNoColumns integers

,student_id,location,age,sql_marks,excel_marks,python_marks,power_bi_marks,english_marks
0,4,Sydney,24,95,99,<NA>,<NA>,75
1,5,Tokyo,24,99,<NA>,89,<NA>,<NA>
2,<NA>,Berlin,22,72,70,99,79,77
3,7,NaN,<NA>,97,90,74,72,85
4,8,NaN,22,91,71,79,80,75
5,<NA>,Toronto,20,93,88,<NA>,93,72
6,10,NaN,18,77,<NA>,100,98,<NA>
7,11,Toronto,21,78,90,88,79,72
8,<NA>,Melbourne,24,80,95,87,90,94
9,13,London,19,97,80,96,<NA>,86


In [5]:
#Fill in empty values

#TODO: add logic to sort id columns from getting messed up

for column in inputTable:
    #For id columns, use forward and backward fill to replace NA
    if "id" in column.lower():
        inputTable[column] = inputTable[column].ffill().bfill()
    #For numeric columns, find the median and fill NA values
    elif column in numericColumns:
        median = inputTable[column].median()
        inputTable[column] = inputTable[column].fillna(median)
    #For categorical columns, find the mode value and fill NA values
    else:
        mode = inputTable[column].mode(dropna=True)
        if not mode.empty:
            inputTable[column] = inputTable[column].fillna(mode[0])

inputTable.head(50)

,student_id,location,age,sql_marks,excel_marks,python_marks,power_bi_marks,english_marks
0,4,Sydney,24,95,99,86,85,75
1,5,Tokyo,24,99,86,89,85,85
2,5,Berlin,22,72,70,99,79,77
3,7,Los Angeles,21,97,90,74,72,85
4,8,Los Angeles,22,91,71,79,80,75
5,8,Toronto,20,93,88,86,93,72
6,10,Los Angeles,18,77,86,100,98,85
7,11,Toronto,21,78,90,88,79,72
8,11,Melbourne,24,80,95,87,90,94
9,13,London,19,97,80,96,85,86
